# Part I: MNIST for ML Beginners
https://www.tensorflow.org/get_started/mnist/beginners

## Load MNIST Dataset

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Start

In [2]:
import tensorflow as tf

### Init placeholders and variables

In [3]:
# Init placeholder for feed
x = tf.placeholder(tf.float32, [None, 28*28])
y_ = tf.placeholder(tf.float32, [None, 10])

# Init Variable to be trained
W = tf.Variable(tf.zeros([28*28, 10]))
b = tf.Variable(tf.zeros([10]))

### Define The Model

In [4]:
y = tf.matmul(x, W) + b
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

### Train Model

In [5]:
batch_size = 100
steps = 1000

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range(steps):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})


### Evaluate Model

In [6]:
correct_predition = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))

In [7]:
accuracy = tf.reduce_mean(tf.cast(correct_predition, tf.float32))

In [8]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.8915


# Part II: Deep MNIST for experts
https://www.tensorflow.org/get_started/mnist/pros

### Build a Multilayer Convolutional Network

Getting 92% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99.2% accuracy -- not state of the art, but respectable.

### Weight Initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons". Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.



In [14]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution and Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

Edwin Comment: (refers to https://www.youtube.com/watch?v=HMcx-zY8JSg)

4 dimensions of strides:
* 1st - image number - has to be 1
* 2nd & 3rd - how many pixels the stride (filter) moves to the right & down each time.
* 4th - channel number - has to be 1 as well

In [15]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

Edwin Comment: (refers to https://www.youtube.com/watch?v=HMcx-zY8JSg)
The 4 dimensions of the CNN weight: 
* 1st & 2nd dimensions are the size of the patch, think of it as a matrix filter, the filter will be used to scan the input image and perform matmul operation to produce the value of the output pixle. 
* 3rd dimension is number of input channels. Think of it as the number of digists required to describe a pixel. In case of gray scale input, we just need one digit. In case of a color input, we need three digits for Red, Green and Blue scale.
* 4th dimension is the number of output channels. Number of patches required = input*output channcels. No idea yet the intuitive reasons. But only difference between the output channels are the result of difference in the randomness of patches. 

Example:
input_image = [
[1, 1, 2, 2],
[3, 3, 4, 4],
[2, 2, 3, 3],
[4, 4, 1, 1]
]

input_channel = 1 # as one pixel only needs one number to represent
output_channel = 2 # let keep it simple. 
patch_size = [2, 2] # 2*2

#cnn_W dimensions = [2, 2, 1, 2], think of it as a 1x2 matrix and each element is a 2x2 filter. 
#in this case, we have 2 filters here, each is of size 2x2. 
#assuming the init value of the two matrics and bias are :

cnn_w1 = [
[0.1, 0.1],
[-0.1, -0.1]
]
cnn_b1 = 0.2

cnn_w2 = [
[0.1, -0.2],
[0.2, -0.1]
]
cnn_b2 = 0.1

#cnn_w1 & cnn_b1 is used to produce output channel one, cnn_w2 and cnn_b2 is for channel two.
#assuming the stride = [1, 1, 1, 1] with zero padding (SAME padding) fill zero when the stride reached outside of the image. 

Applying cnn_w1 & cnn_b1 to the input_image looks like below.
1. [1, 1, 3, 3] x [0.1, 0.1, -0.1, -0.1] + 0.2 = (0.1+0.1-0.3-0.3)+0.2 = -0.2
2. Move 1 step to the right [1, 2, 3, 4] x [0.1, 0.1, -0.1, -0.1] + 0.2 = -0.2
3. Move 1 step to the right [2, 2, 4, 4] x [0.1, 0.1, -0.1, -0.1] + 0.2 = -0.2
4. Move 1 step to the right, padded zeros [2, 0, 4, 0] x [0.1, 0.1, -0.1, -0.1] + 0.2 = 0
5. We now get the first row of output for channcel 1: [-0.2, -0.2, -0.2, 0]
6. Back to the left and move 1 down [3, 3, 2, 2] x [0.1, 0.1, -0.1, -0.1] + 0.2 = 0.4
7. And so on to the right 3 times, we get the 2nd row of out put for channel 1: [0.4, 0.4, 0.4, 0.3]
8. Perfor the same for the rest rows, we get the output for channel 1: [
[-0.2, -0.2, -0.2, 0],
[0.4, 0.4, 0.4, 0.3],
[-0.2, 0.2, 0.6, 0.4], 
[1.0, 0.7, 0.4, 0.3]
]



In [16]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

In [17]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool. The max_pool_2x2 method will reduce the image size to 14x14.

In [19]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer

In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [21]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [22]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.(1)

*(1) For this small convolutional network, performance is actually nearly identical with and without dropout. Dropout is often very effective at reducing overfitting, but it is most useful when training very large neural networks.

In [23]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

Finally, we add a layer, just like for the one layer softmax regression above.

In [24]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Train and Evaluate the Model

How well does this model do? To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network above.

The differences are that:

* We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.
* We will include the additional parameter keep_prob in feed_dict to control the dropout rate.
* We will add logging to every 100th iteration in the training process.

Feel free to go ahead and run this code, but it does 20,000 training iterations and may take a while (possibly up to half an hour), depending on your processor.

In [26]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

for i in range(2000):
  batch = mnist.train.next_batch(50)

  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.14
step 100, training accuracy 0.9
step 200, training accuracy 0.9
step 300, training accuracy 0.92
step 400, training accuracy 0.98
step 500, training accuracy 0.92
step 600, training accuracy 0.94
step 700, training accuracy 0.92
step 800, training accuracy 0.98
step 900, training accuracy 0.98
step 1000, training accuracy 1
step 1100, training accuracy 0.96
step 1200, training accuracy 1
step 1300, training accuracy 0.94
step 1400, training accuracy 0.98
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 0.96
step 1800, training accuracy 0.96
step 1900, training accuracy 1
test accuracy 0.973


The final test set accuracy after running this code should be approximately 99.2%.

We have learned how to quickly and easily build, train, and evaluate a fairly sophisticated deep learning model using TensorFlow.